In [1]:
from models.edenTTS import EdenTTS
from hparams import hparams as hp
from utils.paths import Paths
import torch
import time
import soundfile as sf
import os
from utils.log_util import get_logger
import numpy as np
from text.en_util import text_to_sequence
from qwen_tts import Qwen3TTSTokenizer
import soundfile as sf
# THÊM THƯ VIỆN ĐỂ NGHE AUDIO TRỰC TIẾP TRÊN JUPYTER
from IPython.display import Audio, display 

device = "cuda" if torch.cuda.is_available() else "cpu"
log = get_logger(__name__)

config_file: config/eden.json


2026-02-21 17:26:06,966 INFO en_util.py-25 load symbol2id from resource/symbol_to_id_en.json.
/home/doannv/edentts-with-qwen3-tokenizer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/bin/sh: 1: sox: not found
2026-02-21 17:26:09,736 WARNING __init__.py-10 SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    



********
********
 


In [2]:
# ==========================================
# 1. LOAD TOKENIZER VÀ MODEL (Chỉ cần chạy 1 lần)
# ==========================================
log.info("Loading Qwen3 TTS Tokenizer...")
tokenizer = Qwen3TTSTokenizer.from_pretrained(
    "Qwen/Qwen3-TTS-Tokenizer-12Hz",
    device_map=device,
)

2026-02-21 17:26:09,969 INFO 2871649631.py-4 Loading Qwen3 TTS Tokenizer...


In [13]:
# Khởi tạo và load trọng số EdenTTS
tts_model_id = hp.tts_model_id
paths = Paths(hp.data_path, tts_model_id, speaker=hp.speaker)
tts_model_path = paths.tts_latest_weights
print(f"Đường dẫn checkpoint: {tts_model_path}")

if not os.path.exists(tts_model_path):
    print(f"⚠️ LỖI: Không tìm thấy file checkpoint tại: {tts_model_path}")
else:
    tts_model = EdenTTS(hp).to(device)
    tts_model.load("/home/doannv/edentts-with-qwen3-tokenizer/checkpoints/ljs/tts_eden/step_15000_weights.pyt")
    tts_model.eval() # Bắt buộc set eval mode
    log.info("EdenTTS Model loaded successfully!")

out_path = "./"
os.makedirs(out_path, exist_ok=True)

Đường dẫn checkpoint: checkpoints/ljs/tts_eden/latest_weights.pyt


2026-02-21 17:30:52,550 INFO net_utils.py-25  number of weights:9636480, tol params:116, params requires_grad:115
2026-02-21 17:30:52,551 INFO net_utils.py-25  number of weights:48804864, tol params:87, params requires_grad:87
2026-02-21 17:30:52,551 INFO net_utils.py-25  number of weights:591617, tol params:10, params requires_grad:10
2026-02-21 17:30:52,553 INFO net_utils.py-25  number of weights:65065345, tol params:224, params requires_grad:223
2026-02-21 17:30:52,553 INFO edenTTS.py-92 tol_params: 65065345, text_encoder:9636480,decoder:48804864,dur:591617,tol_infer:59032961
2026-02-21 17:30:52,911 INFO 2425310861.py-13 EdenTTS Model loaded successfully!


In [14]:
enc = tokenizer.encode("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-TTS-Repo/tokenizer_demo_1.wav")
OutputClass = enc.__class__

In [15]:
# ==========================================
# 2. ĐỊNH NGHĨA HÀM SUY LUẬN
# ==========================================
def synthesize_text(text):
    if not os.path.exists(tts_model_path):
        return
        
    log.info(f"Processing: '{text}'")
    phones = text_to_sequence(text, token_type=hp.token_type) 
    phones = torch.tensor(phones).long().unsqueeze(0).to(device)
    
    s1 = time.time()
    # Dự đoán Latent vector liên tục
    mel_pred = tts_model.inference(phones)
    print(mel_pred.shape)
    log.info(f"Acoustic model inference time: {time.time() - s1:.3f}s")
    safe_name = "".join([c if c.isalnum() else "_" for c in text[:15]])
    file_name = f"{safe_name}_{int(time.time())}.wav"
    file_path = os.path.join(out_path, file_name)
    with torch.no_grad():
        # De-normalize: Nhân 2048, làm tròn, ép kiểu integer và kẹp trong khoảng codebook
        tokens = torch.round(mel_pred).long()
        tokens = torch.clamp(tokens, min=0, max=2047)
        new_output = OutputClass(audio_codes=[tokens[0]])
        print(new_output)
        # print({"audio_codes": [tokens[0]]})
        
        # Bọc tensor vào dictionary với key "audio_codes"
        wav, sr = tokenizer.decode(new_output)
        print(wav)
        sf.write("test.wav", wav[0], sr)
    
    log.info(f"Saved at: {file_path}")
    
    # Render thanh Audio ngay trên Jupyter Cell
    display(Audio(wav, rate=sr))

In [18]:
# Ô CHẠY THỬ NGHIỆM (Chạy bao nhiêu lần tùy thích)
text_to_test = "David and Emma looked at each other across the table. The young couple were happy: the food was delicious, the light from the candles was soft and the music was perfect. David looked at Emma's beautiful smile. All the pain of his last break-up disappeared. All his doubts and fears about love had gone. His hand reached out and touched hers. "
synthesize_text(text_to_test)

2026-02-21 17:35:15,529 INFO 21558280.py-8 Processing: 'David and Emma looked at each other across the table. The young couple were happy: the food was delicious, the light from the candles was soft and the music was perfect. David looked at Emma's beautiful smile. All the pain of his last break-up disappeared. All his doubts and fears about love had gone. His hand reached out and touched hers. '
2026-02-21 17:35:15,582 INFO 21558280.py-16 Acoustic model inference time: 0.049s


torch.Size([1, 261, 16])
Qwen3TTSTokenizerV2EncoderOutput(audio_codes=[tensor([[   9, 2002, 1250,  ..., 1136,  790, 1143],
        [1542, 1097, 1296,  ..., 1289,   25,  515],
        [ 862,  302,  540,  ...,  190, 1399,  406],
        ...,
        [1734, 1640, 1700,  ..., 1759,   85, 1978],
        [1734, 1536,  199,  ..., 1860, 1845,  841],
        [ 257,  478,  693,  ..., 1017,  206, 1616]], device='cuda:0')])


2026-02-21 17:35:15,979 INFO 21558280.py-33 Saved at: ./David_and_Emma__1771670115.wav


[array([ 3.3228753e-05,  1.4844978e-05, -6.2632998e-06, ...,
        1.1905354e-04,  1.2247598e-04,  1.0182606e-04],
      shape=(501120,), dtype=float32)]
